In [ ]:
from diffusion_co_design.wfcrl.schema import ScenarioConfig
from diffusion_co_design.wfcrl.design import RandomDesigner
from diffusion_co_design.wfcrl.env import create_env


In [ ]:
scenario = ScenarioConfig(
    n_turbines=10,
    max_steps=150,
    map_x_length=1000,
    map_y_length=1000,
    min_distance_between_turbines=200,
)
designer = RandomDesigner(scenario)
env = create_env(mode="reference", scenario=scenario, designer=designer)

In [ ]:
env._env.state()

In [ ]:
env.reward_shaper